In [1]:
!pip install catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier


pd.set_option('display.max_columns', None)
# 모든 columns를 생략하지 말고 다 보여준다.
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
warnings.filterwarnings('ignore')

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/test.csv')
origin = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/jm1.csv')

In [4]:
columns = ['uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount']

for col in columns:
    origin[col][origin[col] == '?'] = np.nan
    origin[col] = origin[col].astype(float)


In [5]:
train_data = pd.concat([train_data,origin],ignore_index=True)
train_data = train_data.drop_duplicates()

In [6]:
test_data['l'] = 1.0 - test_data['l']
train_data['l'] = 1.0 - train_data['l']

In [7]:
train_data

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0.0,22.0,3.0,1.0,2.0,60.0,278.63,0.94,19.56,14.25,5448.79,0.09,302.71,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1.0,14.0,2.0,1.0,2.0,32.0,151.27,0.86,7.00,21.11,936.71,0.05,52.04,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2.0,11.0,2.0,1.0,2.0,45.0,197.65,0.89,8.05,22.76,1754.01,0.07,97.45,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3.0,8.0,1.0,1.0,1.0,23.0,94.01,0.81,5.25,17.86,473.66,0.03,26.31,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4.0,11.0,2.0,1.0,2.0,17.0,60.94,0.82,5.63,12.44,365.67,0.02,20.31,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112643,NaN,18.0,4.0,1.0,4.0,52.0,241.48,0.86,7.33,32.93,1770.86,0.08,98.38,13,0,2,0,10.0,15.0,30.0,22.0,7.0,False
112644,NaN,9.0,2.0,1.0,2.0,30.0,129.66,0.88,8.25,15.72,1069.68,0.04,59.43,5,0,2,0,12.0,8.0,19.0,11.0,3.0,False
112645,NaN,42.0,4.0,1.0,2.0,103.0,519.57,0.96,26.40,19.68,13716.72,0.17,762.04,29,1,10,0,18.0,15.0,59.0,44.0,7.0,False
112646,NaN,10.0,1.0,1.0,1.0,36.0,147.15,0.88,8.44,17.44,1241.57,0.05,68.98,6,0,2,0,9.0,8.0,21.0,15.0,1.0,False


In [8]:
columns = ['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']
for col in columns:
    print(train_data[col].isnull().sum(),f'{col}의 null 개수 ')

0 loc의 null 개수 
0 v(g)의 null 개수 
0 ev(g)의 null 개수 
0 iv(g)의 null 개수 
0 n의 null 개수 
0 v의 null 개수 
0 l의 null 개수 
0 d의 null 개수 
0 i의 null 개수 
0 e의 null 개수 
0 b의 null 개수 
0 t의 null 개수 
0 lOCode의 null 개수 
0 lOComment의 null 개수 
0 lOBlank의 null 개수 
0 locCodeAndComment의 null 개수 
5 uniq_Op의 null 개수 
5 uniq_Opnd의 null 개수 
5 total_Op의 null 개수 
5 total_Opnd의 null 개수 
5 branchCount의 null 개수 


In [9]:
train_data.drop(['id'],axis=1,inplace=True)
train_data


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,22.0,3.0,1.0,2.0,60.0,278.63,0.94,19.56,14.25,5448.79,0.09,302.71,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.86,7.00,21.11,936.71,0.05,52.04,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.89,8.05,22.76,1754.01,0.07,97.45,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,8.0,1.0,1.0,1.0,23.0,94.01,0.81,5.25,17.86,473.66,0.03,26.31,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,11.0,2.0,1.0,2.0,17.0,60.94,0.82,5.63,12.44,365.67,0.02,20.31,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112643,18.0,4.0,1.0,4.0,52.0,241.48,0.86,7.33,32.93,1770.86,0.08,98.38,13,0,2,0,10.0,15.0,30.0,22.0,7.0,False
112644,9.0,2.0,1.0,2.0,30.0,129.66,0.88,8.25,15.72,1069.68,0.04,59.43,5,0,2,0,12.0,8.0,19.0,11.0,3.0,False
112645,42.0,4.0,1.0,2.0,103.0,519.57,0.96,26.40,19.68,13716.72,0.17,762.04,29,1,10,0,18.0,15.0,59.0,44.0,7.0,False
112646,10.0,1.0,1.0,1.0,36.0,147.15,0.88,8.44,17.44,1241.57,0.05,68.98,6,0,2,0,9.0,8.0,21.0,15.0,1.0,False


In [10]:
train_data.dropna(axis=0,inplace=True)

In [11]:
for col in columns:
    print(train_data[col].isnull().sum(),f'{col}의 null 개수 ')

0 loc의 null 개수 
0 v(g)의 null 개수 
0 ev(g)의 null 개수 
0 iv(g)의 null 개수 
0 n의 null 개수 
0 v의 null 개수 
0 l의 null 개수 
0 d의 null 개수 
0 i의 null 개수 
0 e의 null 개수 
0 b의 null 개수 
0 t의 null 개수 
0 lOCode의 null 개수 
0 lOComment의 null 개수 
0 lOBlank의 null 개수 
0 locCodeAndComment의 null 개수 
0 uniq_Op의 null 개수 
0 uniq_Opnd의 null 개수 
0 total_Op의 null 개수 
0 total_Opnd의 null 개수 
0 branchCount의 null 개수 


In [12]:
train_data['defects'] = train_data['defects'].astype(float)

In [13]:
#train_data.drop(['l'],axis=1,inplace=True) 점수가 더 낮게 나옴

In [14]:
#test_data.drop(['l'],axis=1,inplace=True)

In [15]:
train_data

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,22.0,3.0,1.0,2.0,60.0,278.63,0.94,19.56,14.25,5448.79,0.09,302.71,17,1,1,0,16.0,9.0,38.0,22.0,5.0,0.0
1,14.0,2.0,1.0,2.0,32.0,151.27,0.86,7.00,21.11,936.71,0.05,52.04,11,0,1,0,11.0,11.0,18.0,14.0,3.0,0.0
2,11.0,2.0,1.0,2.0,45.0,197.65,0.89,8.05,22.76,1754.01,0.07,97.45,8,0,1,0,12.0,11.0,28.0,17.0,3.0,0.0
3,8.0,1.0,1.0,1.0,23.0,94.01,0.81,5.25,17.86,473.66,0.03,26.31,4,0,2,0,8.0,6.0,16.0,7.0,1.0,1.0
4,11.0,2.0,1.0,2.0,17.0,60.94,0.82,5.63,12.44,365.67,0.02,20.31,7,0,2,0,7.0,6.0,10.0,10.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112643,18.0,4.0,1.0,4.0,52.0,241.48,0.86,7.33,32.93,1770.86,0.08,98.38,13,0,2,0,10.0,15.0,30.0,22.0,7.0,0.0
112644,9.0,2.0,1.0,2.0,30.0,129.66,0.88,8.25,15.72,1069.68,0.04,59.43,5,0,2,0,12.0,8.0,19.0,11.0,3.0,0.0
112645,42.0,4.0,1.0,2.0,103.0,519.57,0.96,26.40,19.68,13716.72,0.17,762.04,29,1,10,0,18.0,15.0,59.0,44.0,7.0,0.0
112646,10.0,1.0,1.0,1.0,36.0,147.15,0.88,8.44,17.44,1241.57,0.05,68.98,6,0,2,0,9.0,8.0,21.0,15.0,1.0,0.0


In [16]:
def iqr_outlier_detection(data):

    q1, q3 = np.percentile(data, [25, 75])

    iqr = q3 - q1

    threshold = iqr * 1.5
    outliers = [x for x in data if x < q1 - threshold or x > q3 + threshold]

    data_filtered = data[~np.isin(data, outliers)]

    return data_filtered

In [17]:
for col in train_data.columns:
  iqr_outlier_detection(train_data[col])

In [18]:
train_data

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,22.0,3.0,1.0,2.0,60.0,278.63,0.94,19.56,14.25,5448.79,0.09,302.71,17,1,1,0,16.0,9.0,38.0,22.0,5.0,0.0
1,14.0,2.0,1.0,2.0,32.0,151.27,0.86,7.00,21.11,936.71,0.05,52.04,11,0,1,0,11.0,11.0,18.0,14.0,3.0,0.0
2,11.0,2.0,1.0,2.0,45.0,197.65,0.89,8.05,22.76,1754.01,0.07,97.45,8,0,1,0,12.0,11.0,28.0,17.0,3.0,0.0
3,8.0,1.0,1.0,1.0,23.0,94.01,0.81,5.25,17.86,473.66,0.03,26.31,4,0,2,0,8.0,6.0,16.0,7.0,1.0,1.0
4,11.0,2.0,1.0,2.0,17.0,60.94,0.82,5.63,12.44,365.67,0.02,20.31,7,0,2,0,7.0,6.0,10.0,10.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112643,18.0,4.0,1.0,4.0,52.0,241.48,0.86,7.33,32.93,1770.86,0.08,98.38,13,0,2,0,10.0,15.0,30.0,22.0,7.0,0.0
112644,9.0,2.0,1.0,2.0,30.0,129.66,0.88,8.25,15.72,1069.68,0.04,59.43,5,0,2,0,12.0,8.0,19.0,11.0,3.0,0.0
112645,42.0,4.0,1.0,2.0,103.0,519.57,0.96,26.40,19.68,13716.72,0.17,762.04,29,1,10,0,18.0,15.0,59.0,44.0,7.0,0.0
112646,10.0,1.0,1.0,1.0,36.0,147.15,0.88,8.44,17.44,1241.57,0.05,68.98,6,0,2,0,9.0,8.0,21.0,15.0,1.0,0.0


In [19]:
X_train = train_data.drop(['defects'],axis=1)
Y_train = train_data['defects']

In [20]:
X_test = test_data.drop('id',axis=1)

In [21]:
X_test.shape

(67842, 21)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
skf = StratifiedKFold(n_splits = 10, shuffle=True,random_state=42)

In [24]:

params_lgbm = { 'boosting_type' : 'gbdt',
        'objective' : 'binary','colsample_bytree': 1.0,
                'learning_rate': 0.1, 'min_child_samples': 1200,
                'num_leaves': 20, 'reg_alpha': 0.0, 'reg_lambda': 5.0, 'subsample': 1.0}
lgbm = LGBMClassifier(**params_lgbm)
scores = cross_val_score(lgbm,
                         X_train,
                         Y_train,
                         scoring = 'roc_auc',
                         cv = skf,
                         n_jobs = -1)
print(f'{lgbm.__class__.__name__} auroc score : {scores.mean()}')

# without params : : 0.7838295914922109
# with params : 0.785606200470542
#

LGBMClassifier auroc score : 0.7856073724056264


In [ ]:
'''import numpy as np
from sklearn.model_selection import GridSearchCV, KFold


def find_best_hyperparameters(X_train, Y_train):

    # 하이퍼파라미터 후보를 설정합니다.
    hyperparameters = {

        'min_child_samples' : (800,1200),
        'colsample_bytree' : (0.3,1.0),
        'learning_rate' : (0.005,0.1),
        'num_leaves' : (20,60),
        'reg_alpha' : (0.0,10.0),
        'reg_lambda' : (0.0,5.0),
        'subsample' : (0.0,1.0)
    }

    # 교차 검증을 사용하여 하이퍼파라미터를 탐색합니다.
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(LGBMClassifier(), hyperparameters, cv=cv, scoring='roc_auc')
    grid_search.fit(X_train, Y_train)

    # 최적의 하이퍼파라미터를 반환합니다.
    return grid_search.best_params_


# 하이퍼파라미터를 찾습니다.
hyperparameters = find_best_hyperparameters(X_train, Y_train)

# 최적의 하이퍼파라미터를 출력합니다.
print(hyperparameters)'''

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [25]:

cat_params = {'od_type' : 'Iter','eval_metric' : 'AUC',
                         'loss_function' : 'Logloss','bootstrap_type':'Bayesian', "max_bin": 200,'depth': 8, 'l2_leaf_reg': 1.0, 'learning_rate': 0.01, 'random_strength': 0.0}

cat = CatBoostClassifier(verbose = False,**cat_params)
scores = cross_val_score(cat,
                        X_train,
                        Y_train,
                        scoring = 'roc_auc',
                        cv = skf,
                        n_jobs = -1)
print(f'{cat.__class__.__name__} auroc score: {scores.mean()}')

# without params :  0.7829769325555607
# CatBoostClassifier auroc score: 0.7848582234693893
#CatBoostClassifier auroc score: 0.7848582374489382

'''param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }'''


CatBoostClassifier auroc score: 0.7848582374489382


'param = {\n      "random_state":42,\n      \'learning_rate\' : trial.suggest_loguniform(\'learning_rate\', 0.01, 0.3),\n      \'bagging_temperature\' :trial.suggest_loguniform(\'bagging_temperature\', 0.01, 100.00),\n      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),\n      "max_depth":trial.suggest_int("max_depth", 4, 16),\n      \'random_strength\' :trial.suggest_int(\'random_strength\', 0, 100),\n      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),\n      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),\n      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),\n      "max_bin": trial.suggest_int("max_bin", 200, 500),\n      \'od_type\': trial.suggest_categorical(\'od_type\', [\'IncToDec\', \'Iter\']),\n  }'

In [ ]:
'''def find_best_hyperparameters(X_train, Y_train):

   # 하이퍼파라미터 후보를 설정합니다.
  hyperparameters = {
        'learning_rate': (0.01,0.1),
        "depth":(4, 8),
        'random_strength' :(0.0,1.0),
         "l2_leaf_reg": (0.0,1.0)
        #'n_estimators': np.arange(10, 100, 10),
        #'max_leaves' :(10,60),
        #'rsm' : ( 0.0,1.0),
        #"min_child_samples": ( 5, 100),
         }
    # 교차 검증을 사용하여 하이퍼파라미터를 탐색합니다.
  cv = KFold(n_splits=5, shuffle=True, random_state=42)
  grid_search = GridSearchCV(CatBoostClassifier(), hyperparameters, cv=cv, scoring='roc_auc')
  grid_search.fit(X_train, Y_train)
    # 최적의 하이퍼파라미터를 반환합니다.
  return grid_search.best_params_


# 하이퍼파라미터를 찾습니다.
hyperparameters = find_best_hyperparameters(X_train, Y_train)

# 최적의 하이퍼파라미터를 출력합니다.
print(hyperparameters)'''

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1:	learn: 0.5964740	total: 149ms	remaining: 1m 14s
2:	learn: 0.5633496	total: 247ms	remaining: 1m 22s
3:	learn: 0.5382960	total: 339ms	remaining: 1m 24s
4:	learn: 0.5177506	total: 440ms	remaining: 1m 27s
5:	learn: 0.5014248	total: 529ms	remaining: 1m 27s
6:	learn: 0.4887380	total: 629ms	remaining: 1m 29s
7:	learn: 0.4783993	total: 719ms	remaining: 1m 29s
8:	learn: 0.4711999	total: 783ms	remaining: 1m 26s
9:	learn: 0.4649654	total: 882ms	remaining: 1m 27s
10:	learn: 0.4593219	total: 978ms	remaining: 1m 27s
11:	learn: 0.4554440	total: 1.06s	remaining: 1m 27s
12:	learn: 0.4520733	total: 1.17s	remaining: 1m 28s
13:	learn: 0.4493078	total: 1.26s	remaining: 1m 28s
14:	learn: 0.4467550	total: 1.36s	remaining: 1m 29s
15:	learn: 0.4447383	total: 1.46s	remaining: 1m 29s
16:	learn: 0.4433132	total: 1.54s	remaining: 1m 29s
17:	learn: 0.4421586	total: 1.64s	remaining: 1m 29s
18:	learn: 0.4408269	total: 1.75s	remaining: 1m 30s
19:	learn: 0.4397662	total: 1.84s	rem

In [26]:
'''xgb_params = {'n_estimators': 1000,
    'booster': 'gbtree',
    'scale_pos_weight': len(Y_train[Y_train == 0]) / len(Y_train[Y_train == 1]),
    'random_state': 42,'alpha': 0.0, 'colsample_bytree': 1.0,
    'eta': 0.0, 'gamma': 0.0, 'lambda': 0.1, 'learning_rate': 0.2,
    'max_depth': 3, 'min_child_weight': 6, 'subsample': 0.5}'''

xgb_params = {'n_estimators': 1000,
    'learning_rate': 0.01752354328845971,
    'booster': 'gbtree',
    'lambda': 0.08159630121074074,
    'alpha': 0.07564858712175693,
    'subsample': 0.5065979400270813,
    'colsample_bytree': 0.6187340851873067,
    'max_depth': 4,
    'min_child_weight': 5,
    'eta': 0.2603059902806757,
    'gamma': 0.6567360773618207,
    'scale_pos_weight': len(Y_train[Y_train == 0]) / len(Y_train[Y_train == 1]),
    'random_state': 42}
model = xgb.XGBClassifier(**xgb_params)

scores = cross_val_score(model,
                        X_train,
                        Y_train,
                        scoring = 'roc_auc',
                        cv = skf,
                        n_jobs = -1)
print(f'{model.__class__.__name__} auroc score: {scores.mean()}')
# BEFROE CatBoostClassifier auroc score: 0.785644080046026
#AFTER XGBClassifier auroc score: 0.7751595854108592
# XGBClassifier auroc score: 0.7856520669585778 After l change

XGBClassifier auroc score: 0.7856520669585778


In [ ]:
'''from sklearn.model_selection import GridSearchCV, KFold
def find_best_hyperparameters(X_train, Y_train):

   # 하이퍼파라미터 후보를 설정합니다.
  hyperparameters = {
    'learning_rate': (0.0,0.2),
    'lambda': (0.0,0.1),
    'alpha':  (0.0,0.1),
    'subsample': (0.0,0.5),
    'colsample_bytree': (0.5,1.0),
    'max_depth': (3,8),
    'min_child_weight': (4,6),
    'eta': (0.0,1.0),
    'gamma': (0.0,1.0)
        }
    # 교차 검증을 사용하여 하이퍼파라미터를 탐색합니다.
  cv = KFold(n_splits=5, shuffle=True, random_state=42)
  grid_search = GridSearchCV(xgb.XGBClassifier(), hyperparameters, cv=cv, scoring='roc_auc')
  grid_search.fit(X_train, Y_train)
    # 최적의 하이퍼파라미터를 반환합니다.
  return grid_search.best_params_


# 하이퍼파라미터를 찾습니다.
hyperparameters = find_best_hyperparameters(X_train, Y_train)

# 최적의 하이퍼파라미터를 출력합니다.
print(hyperparameters)'''

{'alpha': 0.0, 'colsample_bytree': 1.0, 'eta': 0.0, 'gamma': 0.0, 'lambda': 0.1, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 6, 'subsample': 0.5}


In [27]:
hist_params = {'learning_rate': 0.2, 'max_depth': 4, 'min_samples_leaf': 4}
hist = HistGradientBoostingClassifier(**hist_params)
scores = cross_val_score(hist,
                        X_train,
                        Y_train,
                        scoring = 'roc_auc',
                        cv = skf,
                        n_jobs = -1)
print(f'{hist.__class__.__name__} auroc score: {scores.mean()}')

# with out 0.7843798183172426
# with tuning :  0.784244550618036
#HistGradientBoostingClassifier auroc score: 0.7842147325833738

HistGradientBoostingClassifier auroc score: 0.7842147325833738


In [ ]:
'''from sklearn.model_selection import KFold
def find_best_hyperparameters(X_train, Y_train):

   # 하이퍼파라미터 후보를 설정합니다.
  hyperparameters = {
    'learning_rate': (0.0,0.2),
    'max_depth': (4,12),
    "min_samples_leaf": (4,16),
        }
    # 교차 검증을 사용하여 하이퍼파라미터를 탐색합니다.
  cv = KFold(n_splits=5, shuffle=True, random_state=42)
  grid_search = GridSearchCV(HistGradientBoostingClassifier(), hyperparameters, cv=cv, scoring='roc_auc')
  grid_search.fit(X_train, Y_train)
  return grid_search.best_params_

print(find_best_hyperparameters(X_train, Y_train))'''

{'learning_rate': 0.2, 'max_depth': 4, 'min_samples_leaf': 4}


In [32]:
eclf = VotingClassifier(
     estimators=[('lgbm', lgbm), ('cat', cat), ('xgb', model)],
     voting='soft')
# without this ('hist', hist) VotingClassifier auroc score: 0.7860020854769989,
scores = cross_val_score(eclf,
                        X_train,
                        Y_train,
                        scoring = 'roc_auc',
                        cv = skf,
                        n_jobs = -1)
print(f'{eclf.__class__.__name__} auroc score: {scores.mean()}')

VotingClassifier auroc score: 0.7860020854769989


In [33]:
eclf.fit(X_train,Y_train)


[LightGBM] [Info] Number of positive: 25068, number of negative: 85602
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012346 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3653
[LightGBM] [Info] Number of data points in the train set: 110670, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226511 -> initscore=-1.228117
[LightGBM] [Info] Start training from score -1.228117


VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(min_child_samples=1200,
                                             num_leaves=20, objective='binary',
                                             reg_lambda=5.0)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7d055011cee0>),
                             ('xgb',
                              XGBClassifier(alpha=0.07564858712175693,
                                            base_score=None, booster='gbtree',
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.6...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            lambda=0.08159630121074074,
                                            learning_rate=0.01752354328845971,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None, min_child_weight=5,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, ...))],
                 voting='soft')

In [34]:
submission_df = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/sample_submission.csv')
submission_df.defects = pd.Series(eclf.predict_proba(X_test)[:,1])
submission_df.head()
submission_df.to_csv('/content/drive/MyDrive/캐글 software defects prediction/sample_submission.csv',index=False)
# 만약 NN과 밑에서 합칠려면 마지막 부분은 주석 처리


In [ ]:
'''def contains_bool_values(arr):
    return any(isinstance(x, bool) for x in arr)

contains_bool_values(Y_train)'''

'def contains_bool_values(arr):\n    return any(isinstance(x, bool) for x in arr)\n\ncontains_bool_values(Y_train)'

In [ ]:
submission_df

,id,defects
0,101763,0.324696
1,101764,0.248621
2,101765,0.698134
3,101766,0.526980
4,101767,0.200590
...,...,...
67837,169600,0.347051
67838,169601,0.173378
67839,169602,0.241483
67840,169603,0.144685


## 여기서부터 nn모델입니다

In [ ]:
skf = StratifiedKFold(n_splits = 5, shuffle=True,random_state=42)

models = []
auc_scores = []
test_predictions = np.zeros(len(X_test))

for fold,(train_index,val_index) in enumerate(skf.split(X_train,Y_train)):
    X_train_fold,X_val_fold = X_train[train_index],X_train[val_index]
    y_train_fold,y_val_fold = Y_train.iloc[train_index],Y_train.iloc[val_index]

    model1 = Sequential()
    model1.add(Dense(64,input_shape = (X_train_fold.shape[1],),activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(32,activation='relu'))
    model1.add(Dense(1,activation='sigmoid'))

    model1.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy',tf.keras.metrics.AUC()])
    early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights = True)
    class_weights = {0:1, 1:len(y_train_fold[y_train_fold==0]) / len(y_train_fold[y_train_fold == 1])}

    epochs = 70
    batch_size = 32
    progress_bar = tqdm(range(epochs),position=0,desc = 'Fold Progress')

    for epoch in progress_bar:
        history = model1.fit(X_train_fold,y_train_fold,epochs=1,
                           batch_size=batch_size,class_weight=class_weights,
                           verbose=0,
                           validation_data=(X_val_fold,y_val_fold),
                           callbacks=[early_stopping])
        progress_bar.set_description(f'Epoch {epoch+1}/{epochs} - Loss: {history.history["loss"][0]:.4f} - Accuracy: {history.history["accuracy"][0]:.4f}')
        #progress_bar.set_description(f'Epoch {epoch + 1}/{epochs} - Loss: {history.history['loss'][0] : .4f} - Accuracy:{history.history['accuracy'][0] : .4f}')

    y_pred_prob = model1.predict(X_val_fold)
    auc = roc_auc_score(y_val_fold,y_pred_prob)
    auc_scores.append(auc)

    test_predictions += model1.predict(X_test).flatten()







average_auc = sum(auc_scores) / len(auc_scores)
print(f'Average AUC score : {average_auc:.4f}')

test_predictions /= skf.n_splits

print('Saved models:')
print(models)



Fold Progress:   0%|          | 0/70 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
submission_df.defects += test_predictions.reshape(-1)


In [ ]:
submission_df.defects /= 2

In [ ]:
submission_df.to_csv('/content/drive/MyDrive/캐글 software defects prediction/sample_submission.csv',index=False)